In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import GNB

iris = load_iris()
X = iris.data
y = iris.target

# 分訓練 / 測試
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 建立模型
model = GaussianNaiveBayes()
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 顯示準確率
acc = (y_pred == y_test).mean()
print("模型準確率:", acc)

# 顯示部分預測
print("前十筆預測：", y_pred[:10])
print("前十筆真值：", y_test[:10])

模型準確率: 0.9777777777777777
前十筆預測： [1 0 2 1 1 0 1 2 1 1]
前十筆真值： [1 0 2 1 1 0 1 2 1 1]


`GNB.py` 檔案很可能包含了一個自定義的 Gaussian Naive Bayes 模型實作。在許多機器學習的案例中，開發者可能會選擇從頭開始實現一個模型，或者對現有的庫進行客製化。這個檔案通常會定義一個名為 `GaussianNaiveBayes` 的類別，其中包含了訓練模型 (fit 方法) 和進行預測 (predict 方法) 的邏輯。

在機器學習中，`fit` 方法是用來訓練模型的關鍵步驟。對於一個像 `GaussianNaiveBayes` 這樣的模型，`fit(X_train, y_train)` 方法會執行以下操作：

1.  **學習資料的統計特性**：
    *   對於每個類別 (在鳶尾花數據集中是3個品種)，模型會計算每個特徵 (如花萼長度、花萼寬度等) 的平均值 (mean) 和標準差 (standard deviation)。
    *   這些統計量是假設每個特徵在每個類別內都服從高斯 (正態) 分佈的基礎。

2.  **計算類別先驗機率**：
    *   模型會計算每個類別在訓練資料中出現的頻率，也就是每個類別的先驗機率 (prior probability)。例如，如果訓練資料中類別0有50筆、類別1有50筆、類別2有50筆，那麼每個類別的先驗機率都是 1/3。

3.  **儲存學習到的參數**：
    *   訓練完成後，模型會儲存這些學習到的統計特性 (每個類別下每個特徵的平均值和標準差) 和類別先驗機率。這些參數就是模型從訓練資料中「學到」的知識，後續將用於預測新的資料點。

簡而言之，`fit` 方法的目標是讓模型從提供的訓練資料 (`X_train` 為特徵，`y_train` 為目標標籤) 中學習內部參數，以便它能夠理解資料的模式，並在之後對未見過的資料進行預測。

## Gaussian Naive Bayes (高斯樸素貝葉斯) 演算法說明

高斯樸素貝葉斯是一種基於貝葉斯定理的分類演算法，它假設每個特徵在給定類別的條件下是相互獨立的（這就是「樸素」的由來）。當特徵的條件機率分佈是高斯分佈（正態分佈）時，我們稱之為高斯樸素貝葉斯。

其核心思想是根據貝葉斯定理，計算每個類別的後驗機率，並選擇後驗機率最高的類別作為預測結果：

$$P(C_k | \mathbf{x}) = \frac{P(\mathbf{x} | C_k) P(C_k)}{P(\mathbf{x})}$$

其中：
*   $P(C_k | \mathbf{x})$ 是在給定特徵向量 $\mathbf{x}$ 的情況下，數據點屬於類別 $C_k$ 的後驗機率。
*   $P(\mathbf{x} | C_k)$ 是在給定類別 $C_k$ 的情況下，特徵向量 $\mathbf{x}$ 出現的機率 (似然度)。
*   $P(C_k)$ 是類別 $C_k$ 的先驗機率。
*   $P(\mathbf{x})$ 是特徵向量 $\mathbf{x}$ 的證據機率（通常在比較不同類別時可以省略，因為它是常數）。

由於「樸素」的假設，即特徵之間是相互獨立的，所以 $P(\mathbf{x} | C_k)$ 可以寫成各個特徵條件機率的乘積：

$$P(\mathbf{x} | C_k) = P(x_1 | C_k) \cdot P(x_2 | C_k) \cdot \ldots \cdot P(x_n | C_k)$$

對於高斯樸素貝葉斯，每個 $P(x_i | C_k)$ 都是由高斯機率密度函數給出的：

$$P(x_i | C_k) = \frac{1}{\sqrt{2\pi\sigma_{ki}^2}} \exp\left(-\frac{(x_i - \mu_{ki})^2}{2\sigma_{ki}^2}\right)$$

其中，$\mu_{ki}$ 和 $\sigma_{ki}^2$ 分別是類別 $C_k$ 中特徵 $x_i$ 的平均值和變異數。

## `GNB.py` 中的實作說明

基於上述演算法，`GNB.py` 檔案中 `GaussianNaiveBayes` 類別的實作通常會包含以下兩個主要方法：

### `fit(X_train, y_train)` 方法

這個方法負責模型的訓練，也就是從訓練資料中學習演算法所需的參數。具體來說，它會執行：

1.  **計算類別先驗機率 (`P(C_k)`)**：遍歷訓練標籤 `y_train`，計算每個類別 $C_k$ 在總訓練樣本數中所佔的比例。這些值會被儲存起來，例如儲存在一個字典或陣列中。

2.  **計算每個類別下每個特徵的統計參數 (`\mu_{ki}` 和 `\sigma_{ki}^2`)**：
    *   將訓練資料 `X_train` 根據其對應的類別 `y_train` 進行分組。
    *   對於每個類別 $C_k$ 和每個特徵 $x_i$，計算該類別中所有樣本的特徵 $x_i$ 的平均值 ($\mu_{ki}$) 和變異數 ($\sigma_{ki}^2$)。
    *   這些平均值和變異數會被儲存起來，作為模型預測時計算似然度 `P(x_i | C_k)` 的依據。

這些學習到的參數（類別先驗機率、每個類別下每個特徵的平均值和變異數）構成了訓練好的模型，它們在 `fit` 方法結束後會作為模型物件的內部狀態被保存。

### `predict(X_test)` 方法

這個方法負責根據訓練好的模型對新的資料進行預測。對於測試集中的每個樣本 $\mathbf{x}_{new}$，它會執行：

1.  **對於每個類別 $C_k$**：
    *   **計算似然度 $P(\mathbf{x}_{new} | C_k)$**：
        *   使用在 `fit` 階段學習到的 $\mu_{ki}$ 和 $\sigma_{ki}^2$，對 $X_{new}$ 中的每個特徵 $x_{new,i}$ 計算其在類別 $C_k$ 下的高斯機率密度 $P(x_{new,i} | C_k)$。
        *   將所有特徵的條件機率相乘，得到 $P(\mathbf{x}_{new} | C_k)$。
    *   **計算後驗機率 $P(C_k | \mathbf{x}_{new})$ 的分子**：將計算出的似然度 $P(\mathbf{x}_{new} | C_k)$ 乘以該類別的先驗機率 $P(C_k)$。

2.  **做出預測**：比較所有類別計算出的後驗機率分子，選擇具有最大值的類別作為該樣本 $\mathbf{x}_{new}$ 的預測結果。這個過程對 `X_test` 中的每個樣本重複，最終返回一個包含所有預測類別的陣列。